In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [11]:
filepath = r"C:\Users\ram.gv\Desktop\GH\FPL-Lineup-Prediction\data\converted_data\gameweek_data.csv"

df = pd.read_csv(filepath,index_col=0)


In [12]:
df.head()

,player_id,player_name,now_cost,position,team,gw1,gw2,gw3,gw4,gw5,gw6,gw7,gw8,gw9,gw10,gw11,gw12
0,1,Raya,59,GKP,Arsenal,10.0,6.0,2.0,6.0,2.0,2.0,6.0,6.0,6.0,6.0,1.0,2
1,2,Arrizabalaga,42,GKP,Arsenal,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
2,3,Hein,40,GKP,Arsenal,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
3,4,Setford,40,GKP,Arsenal,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
4,5,Gabriel,64,DEF,Arsenal,6.0,6.0,2.0,9.0,2.0,13.0,9.0,12.0,9.0,12.0,1.0,0


In [17]:
gw_cols = [f"gw{i}" for i in range(1,13)]
df["avg_pt"] = df[gw_cols].mean(axis = 1).round(2)

In [24]:
df.loc[df['avg_pt'].idxmax()]

player_id           430
player_name     Haaland
now_cost            149
position            FWD
team           Man City
gw1                13.0
gw2                 2.0
gw3                 9.0
gw4                13.0
gw5                 9.0
gw6                16.0
gw7                 8.0
gw8                13.0
gw9                 2.0
gw10               13.0
gw11                4.0
gw12                  2
avg_pt             8.67
Name: 429, dtype: object

In [28]:
n = len(gw_cols)
weights = np.arange(1,n+1)

weights = weights / weights.sum()

df['weighted_avg'] = df[gw_cols].values.dot(weights).round(2)

In [29]:
df.head()

,player_id,player_name,now_cost,position,team,gw1,gw2,gw3,gw4,gw5,gw6,gw7,gw8,gw9,gw10,gw11,gw12,avg_pt,weighted_avg
0,1,Raya,59,GKP,Arsenal,10.0,6.0,2.0,6.0,2.0,2.0,6.0,6.0,6.0,6.0,1.0,2,4.58,4.01
1,2,Arrizabalaga,42,GKP,Arsenal,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.00,0.00
2,3,Hein,40,GKP,Arsenal,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.00,0.00
3,4,Setford,40,GKP,Arsenal,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.00,0.00
4,5,Gabriel,64,DEF,Arsenal,6.0,6.0,2.0,9.0,2.0,13.0,9.0,12.0,9.0,12.0,1.0,0,6.75,6.65


In [32]:
top_10 = df.nlargest(10, 'weighted_avg')
print(top_10)

     player_id player_name  now_cost position            team   gw1  gw2  gw3  \
429        430     Haaland       149      FWD        Man City  13.0  2.0  9.0   
255        256       Muñoz        59      DEF  Crystal Palace   5.0  5.0  9.0   
4            5     Gabriel        64      DEF         Arsenal   6.0  6.0  2.0   
265        266         Eze        77      MID         Arsenal   3.0  0.0  1.0   
20          21        Rice        70      MID         Arsenal   6.0  6.0  4.0   
294        295       Keane        45      DEF         Everton   2.0  8.0  3.0   
19          20    Trossard        69      MID         Arsenal   0.0  1.0  0.0   
235        236        Neto        71      MID         Chelsea   3.0  7.0  3.0   
135        136      Thiago        66      FWD       Brentford   6.0  2.0  6.0   
487        488    Bruno G.        66      MID       Newcastle   3.0  6.0  3.0   

      gw4  gw5   gw6   gw7   gw8   gw9  gw10  gw11  gw12  avg_pt  weighted_avg  
429  13.0  9.0  16.0   8.0 